<a href="https://colab.research.google.com/github/MatiasSiles/Sales-Optimization/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I will apply feature engineering, data cleaning, feature selection, etc. For all prepare train the models

In [36]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('/content/Sales_Business.csv')

#Data cleaning for ML models

i not consider "return_reason" for data cleaning because it has 11156 nan, that's a lot but for other side, is a important feature for customer classification. Then, in other situations more information about this column could be requested

In [37]:
important_features_impute_mean = ['customer_age', 'satisfaction_score']

for col in important_features_impute_mean:
    df[col].fillna(df[col].mean(), inplace=True)

df['customer_gender'] = df['customer_gender'].fillna("unknown")

df.drop('return_reason', axis=1, inplace=True)

#Feature Engineering

In [38]:
# Copy for feature engineering
df_fe = df.copy()

In [ ]:
# 1. Date-based features
df_fe['date'] = pd.to_datetime(df_fe['date'])
df_fe['days_since_start'] = (df_fe['date'] - df_fe['date'].min()).dt.days
df_fe['week_of_year'] = df_fe['date'].dt.isocalendar().week # the week number of the date, e.g: 2021-08-30 is the week 35 in 2021
df_fe['is_month_start'] = df_fe['date'].dt.is_month_start.astype(int) # 1 = if the date is the first day of the month
df_fe['is_month_end'] = df_fe['date'].dt.is_month_end.astype(int)
df_fe['is_quarter_start'] = df_fe['date'].dt.is_quarter_start.astype(int)
df_fe['is_quarter_end'] = df_fe['date'].dt.is_quarter_end.astype(int)
df_fe

In [40]:
# 2. Customer-based features
customer_stats = df_fe.groupby('customer_id').agg({
    'order_id': 'count',
    'total_amount': ['sum', 'mean'],
    'profit': 'sum',
    'date': ['min', 'max']
})
customer_stats.columns = ['order_frequency', 'total_spent', 'avg_order_value',
                          'total_profit_generated', 'first_purchase', 'last_purchase']

# Calculate recency (days since last purchase)
customer_stats['recency_days'] = (df_fe['date'].max() - customer_stats['last_purchase']).dt.days
customer_stats['customer_lifetime_days'] = (customer_stats['last_purchase'] - customer_stats['first_purchase']).dt.days

In [41]:
# df_fe = df_fe.merge(customer_stats, left_on='customer_id', right_index=True, how='left')
df_fe = df_fe.merge(customer_stats, on='customer_id', how='left')

In [32]:
# 3. Product-based features
# Product performance metrics
product_stats = df_fe.groupby('product_name').agg({
    'quantity': 'sum',
    'total_amount': 'sum',
    'profit_margin': 'mean',
    'is_returned': 'mean'
})
product_stats.columns = ['product_total_qty_sold', 'product_total_revenue',
                        'product_avg_margin', 'product_return_rate']

In [ ]:
df_fe = df_fe.merge(product_stats, on='product_name', how='left')

In [ ]:
# 4. Sales representative performance
rep_stats = df_fe.groupby('sales_rep').agg({
    'total_amount': ['sum', 'mean'],
    'profit': 'sum',
    'satisfaction_score': 'mean'
})
rep_stats.columns = ['rep_total_sales', 'rep_avg_order_value',
                    'rep_total_profit', 'rep_avg_satisfaction']

In [ ]:
df_fe = df_fe.merge(rep_stats, on='sales_rep', how='left')

In [ ]:
# 5. Discount and pricing features
df_fe['discount_impact'] = df_fe['discount_amount'] / df_fe['subtotal']
df_fe['effective_price'] = df_fe['unit_price'] * (1 - df_fe['discount_rate'])
df_fe['price_per_profit_ratio'] = df_fe['unit_price'] / (df_fe['profit'] + 0.01)  # Avoid division by zero
df_fe['is_high_discount'] = (df_fe['discount_rate'] > df_fe['discount_rate'].quantile(0.75)).astype(int)
df_fe['discount_category'] = pd.cut(df_fe['discount_rate'],
                                    bins=[0, 0.05, 0.15, 0.25, 1.0],
                                    labels=['No_Discount', 'Low_Discount', 'Medium_Discount', 'High_Discount'])

In [ ]:
# 6. Order complexity features
df_fe['order_complexity'] = df_fe['quantity'] * df_fe.groupby('order_id')['product_name'].transform('nunique')
df_fe['is_bulk_order'] = (df_fe['quantity'] > df_fe['quantity'].quantile(0.8)).astype(int)
df_fe['shipping_to_total_ratio'] = df_fe['shipping_cost'] / df_fe['total_amount']

In [ ]:
# 7. Seasonal and cyclical features
df_fe['is_holiday_season'] = ((df_fe['month'] == 12) | (df_fe['month'] == 1)).astype(int)
df_fe['is_summer_season'] = ((df_fe['month'] >= 6) & (df_fe['month'] <= 8)).astype(int)
df_fe['is_back_to_school'] = ((df_fe['month'] == 8) | (df_fe['month'] == 9)).astype(int)

In [ ]:
# 8. Risk and quality indicators
df_fe['customer_risk_score'] = (
    (df_fe['recency_days'] > df_fe['recency_days'].quantile(0.75)) * 0.3 +
    (df_fe['satisfaction_score'] < df_fe['satisfaction_score'].quantile(0.25)) * 0.4 +
    (df_fe['is_returned'] == 1) * 0.3
)

In [ ]:
# 9. Encoding categorical variables
# One-hot encoding for low cardinality categorical variables
categorical_to_encode = ['customer_gender', 'customer_segment', 'sales_channel',
                        'payment_method', 'day_of_week', 'discount_category']

for col in categorical_to_encode:
    if col in df_fe.columns:
        dummies = pd.get_dummies(df_fe[col], prefix=col, drop_first=True)
        df_fe = pd.concat([df_fe, dummies], axis=1)

# Target encoding for high cardinality categorical variables
high_cardinality_cols = ['product_category', 'product_name', 'region', 'sales_rep', 'lead_source']
target_col = 'total_amount'  # Can be changed based on prediction target

for col in high_cardinality_cols:
    if col in df_fe.columns:
        target_mean = df_fe.groupby(col)[target_col].mean()
        df_fe[f'{col}_target_encoded'] = df_fe[col].map(target_mean)

In [33]:
# 10. Interaction features
df_fe['age_segment_interaction'] = df_fe['customer_age'] * df_fe['customer_segment_Premium']
df_fe['price_quantity_interaction'] = df_fe['unit_price'] * df_fe['quantity']
df_fe['discount_satisfaction_interaction'] = df_fe['discount_rate'] * df_fe['satisfaction_score']

print(f"Original features: {df.shape[1]}")
print(f"Features after engineering: {df_fe.shape[1]}")
print(f"New features created: {df_fe.shape[1] - df.shape[1]}")

Original features: 31
Features after engineering: 97
New features created: 66


In [34]:
# ========================================
# 7. FEATURE ENGINEERING
# ========================================

def feature_engineering(df):
    """Comprehensive feature engineering for predictive modeling"""

    print("=== FEATURE ENGINEERING ===")

    # Create a copy for feature engineering
    df_fe = df.copy()

    # 1. Date-based features
    df_fe['date'] = pd.to_datetime(df_fe['date'])
    df_fe['days_since_start'] = (df_fe['date'] - df_fe['date'].min()).dt.days
    df_fe['week_of_year'] = df_fe['date'].dt.isocalendar().week
    df_fe['is_month_start'] = df_fe['date'].dt.is_month_start.astype(int)
    df_fe['is_month_end'] = df_fe['date'].dt.is_month_end.astype(int)
    df_fe['is_quarter_start'] = df_fe['date'].dt.is_quarter_start.astype(int)
    df_fe['is_quarter_end'] = df_fe['date'].dt.is_quarter_end.astype(int)

    # 2. Customer-based features
    # Customer frequency and recency
    customer_stats = df_fe.groupby('customer_id').agg({
        'order_id': 'count',
        'total_amount': ['sum', 'mean'],
        'profit': 'sum',
        'date': ['min', 'max']
    })
    customer_stats.columns = ['order_frequency', 'total_spent', 'avg_order_value',
                             'total_profit_generated', 'first_purchase', 'last_purchase']

    # Calculate recency (days since last purchase)
    customer_stats['recency_days'] = (df_fe['date'].max() - customer_stats['last_purchase']).dt.days
    customer_stats['customer_lifetime_days'] = (customer_stats['last_purchase'] - customer_stats['first_purchase']).dt.days

    # Merge back to main dataset
    df_fe = df_fe.merge(customer_stats, left_on='customer_id', right_index=True, how='left')

    # 3. Product-based features
    # Product performance metrics
    product_stats = df_fe.groupby('product_name').agg({
        'quantity': 'sum',
        'total_amount': 'sum',
        'profit_margin': 'mean',
        'is_returned': 'mean'
    })
    product_stats.columns = ['product_total_qty_sold', 'product_total_revenue',
                            'product_avg_margin', 'product_return_rate']

    df_fe = df_fe.merge(product_stats, left_on='product_name', right_index=True, how='left')

    # 4. Sales representative performance
    rep_stats = df_fe.groupby('sales_rep').agg({
        'total_amount': ['sum', 'mean'],
        'profit': 'sum',
        'satisfaction_score': 'mean'
    })
    rep_stats.columns = ['rep_total_sales', 'rep_avg_order_value',
                        'rep_total_profit', 'rep_avg_satisfaction']

    df_fe = df_fe.merge(rep_stats, left_on='sales_rep', right_index=True, how='left')

    # 5. Discount and pricing features
    df_fe['discount_impact'] = df_fe['discount_amount'] / df_fe['subtotal']
    df_fe['effective_price'] = df_fe['unit_price'] * (1 - df_fe['discount_rate'])
    df_fe['price_per_profit_ratio'] = df_fe['unit_price'] / (df_fe['profit'] + 0.01)  # Avoid division by zero
    df_fe['is_high_discount'] = (df_fe['discount_rate'] > df_fe['discount_rate'].quantile(0.75)).astype(int)
    df_fe['discount_category'] = pd.cut(df_fe['discount_rate'],
                                       bins=[0, 0.05, 0.15, 0.25, 1.0],
                                       labels=['No_Discount', 'Low_Discount', 'Medium_Discount', 'High_Discount'])

    # 6. Order complexity features
    df_fe['order_complexity'] = df_fe['quantity'] * df_fe.groupby('order_id')['product_name'].transform('nunique')
    df_fe['is_bulk_order'] = (df_fe['quantity'] > df_fe['quantity'].quantile(0.8)).astype(int)
    df_fe['shipping_to_total_ratio'] = df_fe['shipping_cost'] / df_fe['total_amount']

    # 7. Seasonal and cyclical features
    df_fe['is_holiday_season'] = ((df_fe['month'] == 12) | (df_fe['month'] == 1)).astype(int)
    df_fe['is_summer_season'] = ((df_fe['month'] >= 6) & (df_fe['month'] <= 8)).astype(int)
    df_fe['is_back_to_school'] = ((df_fe['month'] == 8) | (df_fe['month'] == 9)).astype(int)

    # 8. Risk and quality indicators
    df_fe['customer_risk_score'] = (
        (df_fe['recency_days'] > df_fe['recency_days'].quantile(0.75)) * 0.3 +
        (df_fe['satisfaction_score'] < df_fe['satisfaction_score'].quantile(0.25)) * 0.4 +
        (df_fe['is_returned'] == 1) * 0.3
    )

    # 9. Encoding categorical variables
    # One-hot encoding for low cardinality categorical variables
    categorical_to_encode = ['customer_gender', 'customer_segment', 'sales_channel',
                           'payment_method', 'day_of_week', 'discount_category']

    for col in categorical_to_encode:
        if col in df_fe.columns:
            dummies = pd.get_dummies(df_fe[col], prefix=col, drop_first=True)
            df_fe = pd.concat([df_fe, dummies], axis=1)

    # Target encoding for high cardinality categorical variables
    high_cardinality_cols = ['product_category', 'product_name', 'region', 'sales_rep', 'lead_source']
    target_col = 'total_amount'  # Can be changed based on prediction target

    for col in high_cardinality_cols:
        if col in df_fe.columns:
            target_mean = df_fe.groupby(col)[target_col].mean()
            df_fe[f'{col}_target_encoded'] = df_fe[col].map(target_mean)

    # 10. Interaction features
    df_fe['age_segment_interaction'] = df_fe['customer_age'] * df_fe['customer_segment_Premium']
    df_fe['price_quantity_interaction'] = df_fe['unit_price'] * df_fe['quantity']
    df_fe['discount_satisfaction_interaction'] = df_fe['discount_rate'] * df_fe['satisfaction_score']

    print(f"Original features: {df.shape[1]}")
    print(f"Features after engineering: {df_fe.shape[1]}")
    print(f"New features created: {df_fe.shape[1] - df.shape[1]}")

    return df_fe

# ========================================
# 8. FEATURE SELECTION AND PREPARATION
# ========================================

def prepare_features_for_modeling(df_fe):
    """Prepare features for machine learning models"""

    print("=== FEATURE PREPARATION FOR MODELING ===")

    # Remove features not suitable for modeling
    columns_to_drop = ['order_id', 'customer_id', 'product_name', 'date',
                      'first_purchase', 'last_purchase', 'sales_rep']

    # Create modeling dataset
    df_model = df_fe.drop(columns=[col for col in columns_to_drop if col in df_fe.columns])

    # Handle remaining missing values
    # Numerical columns: fill with median
    numerical_cols = df_model.select_dtypes(include=[np.number]).columns
    df_model[numerical_cols] = df_model[numerical_cols].fillna(df_model[numerical_cols].median())

    # Categorical columns: fill with mode
    categorical_cols = df_model.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        df_model[col] = df_model[col].fillna(df_model[col].mode()[0])

    # Feature importance analysis using correlation with target variables
    target_variables = ['total_amount', 'profit', 'satisfaction_score', 'is_returned']

    print("TOP 20 FEATURES BY CORRELATION WITH TARGETS:")
    for target in target_variables:
        if target in df_model.columns:
            correlations = df_model.corr()[target].abs().sort_values(ascending=False)
            print(f"\nTop features correlated with {target}:")
            print(correlations.head(10))

    print(f"\nFinal dataset shape for modeling: {df_model.shape}")
    print(f"Features ready for ML: {df_model.shape[1]}")

    return df_model